# 🧬 Geometry-Complete Equivariant Diffusion Model
## De Novo Drug Design Training Notebook

**Requirements:** GPU Runtime (T4), ~50GB disk for real data

## Cell 1: Check GPU

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU! Runtime > Change runtime type > GPU")

## Cell 2: Install Dependencies

In [ ]:
!pip install -q torch-geometric rdkit scipy numpy pyyaml tqdm wandb
from rdkit import Chem
print("✅ All dependencies installed")

## Cell 3: Clone Repository

In [ ]:
!git clone https://github.com/Nethrananda21/geom_diffusion.git
%cd geom_diffusion
!git pull origin master

## Cell 4: Download CrossDocked2020 Dataset (~50GB)
⚠️ Skip this cell to use synthetic data for quick testing

In [ ]:
# Download CrossDocked2020 (takes ~30 min)
!wget --no-check-certificate -q --show-progress http://bits.csb.pitt.edu/files/crossdock2020/CrossDocked2020_v1.3.tgz

# Extract
!mkdir -p data
!tar -xzf CrossDocked2020_v1.3.tgz -C ./data/
print("✅ Dataset extracted")

# Clean up archive to save space
!rm CrossDocked2020_v1.3.tgz

## Cell 5: Preprocess Dataset

In [ ]:
# Preprocess: creates train_data.pkl and val_data.pkl
!python preprocess_crossdocked.py \
    --data_dir ./data/CrossDocked2020 \
    --output_dir ./data/crossdocked \
    --config configs/debug_t4.yaml

print("\n✅ Preprocessing complete")
!ls -la data/crossdocked/

## Cell 6: ⚠️ DELETE OLD CACHE (IMPORTANT!)

In [ ]:
# CRITICAL: Delete old synthetic cache to use real data
import shutil
from pathlib import Path

cache_dir = Path('data/cache')
if cache_dir.exists():
    shutil.rmtree(cache_dir)
    print("🗑️ Deleted old cache")
else:
    print("ℹ️ No cache to delete")

# Verify real data exists
train_pkl = Path('data/crossdocked/train_data.pkl')
val_pkl = Path('data/crossdocked/val_data.pkl')

if train_pkl.exists() and val_pkl.exists():
    print(f"✅ Real data ready:")
    print(f"   - {train_pkl} ({train_pkl.stat().st_size / 1e6:.1f} MB)")
    print(f"   - {val_pkl} ({val_pkl.stat().st_size / 1e6:.1f} MB)")
else:
    print("⚠️ Real data NOT found - will use synthetic")

## Cell 7: Configure Training

In [ ]:
import yaml

with open('configs/debug_t4.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Adjust for Colab
config['training']['max_epochs'] = 10  # Quick test
config['hardware']['num_workers'] = 2  # Colab limit

with open('configs/debug_t4.yaml', 'w') as f:
    yaml.dump(config, f)

print(f"✅ Config: batch_size={config['training']['batch_size']}, epochs={config['training']['max_epochs']}")

## Cell 8: Start Training 🚀

In [ ]:
# Train the model
# Watch the logs - should say "Loading preprocessed data" NOT "SYNTHETIC"
!python train.py --config configs/debug_t4.yaml --checkpoint_dir ./checkpoints

## Cell 9: Resume Training (If Interrupted)

In [ ]:
# Uncomment to resume from checkpoint
# !python train.py --config configs/debug_t4.yaml --resume ./checkpoints/best_model.pt

## Cell 10: Download Trained Model

In [ ]:
from google.colab import files
from pathlib import Path

ckpt = Path('checkpoints/best_model.pt')
if ckpt.exists():
    files.download(str(ckpt))
    print("✅ Model downloaded")
else:
    print("❌ No checkpoint yet")